In [1]:
import tensorflow as tf
from tensorflow.keras import layers, losses

# 1. Load the dataset (Sentiment analysis: Positive or Negative)
vocab_size = 10000
sequence_length = 250

(train_data, train_labels), (test_data, test_labels) = tf.keras.datasets.imdb.load_data(num_words=vocab_size)

# 2. Preprocess: Pad sequences so every input has the same length
train_data = tf.keras.preprocessing.sequence.pad_sequences(train_data, maxlen=sequence_length)
test_data = tf.keras.preprocessing.sequence.pad_sequences(test_data, maxlen=sequence_length)

# 3. Build the Model
model = tf.keras.Sequential([
    layers.Embedding(vocab_size, 64),
    layers.Bidirectional(layers.LSTM(64, return_sequences=True)),
    layers.Bidirectional(layers.LSTM(32)),
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam',
              loss=losses.BinaryCrossentropy(),
              metrics=['accuracy'])

# 4. Train the Model
history = model.fit(train_data, train_labels, epochs=10, 
                    validation_split=0.2, batch_size=512)

# 5. Evaluate
results = model.evaluate(test_data, test_labels)
print(f"Test Loss: {results[0]}, Test Accuracy: {results[1]}")

# 6. Save the project to upload (h5 or SavedModel format)
model.save('text_classification_model.h5')

Epoch 1/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 144s 3s/step - accuracy: 0.6229 - loss: 0.6390 - val_accuracy: 0.7976 - val_loss: 0.4674
Epoch 2/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 168s 4s/step - accuracy: 0.8399 - loss: 0.3929 - val_accuracy: 0.8448 - val_loss: 0.3738
Epoch 3/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 207s 4s/step - accuracy: 0.8946 - loss: 0.2840 - val_accuracy: 0.8472 - val_loss: 0.3597
Epoch 4/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 210s 5s/step - accuracy: 0.9273 - loss: 0.2061 - val_accuracy: 0.8798 - val_loss: 0.3323
Epoch 5/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 187s 5s/step - accuracy: 0.9499 - loss: 0.1554 - val_accuracy: 0.8720 - val_loss: 0.3645
Epoch 6/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 205s 5s/step - accuracy: 0.9682 - loss: 0.1056 - val_accuracy: 0.8462 - val_loss: 0.4573
Epoch 7/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 188s 5s/step - accuracy: 0.9589 - loss: 0.1239 - val_accuracy: 0.8544 - val_loss: 0.4780
Epoch 8/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 200s 5s/step - accuracy: 0.9778 - loss: 0.0784 - val_accuracy: 0.8346 - v

Test Loss: 0.6052758693695068, Test Accuracy: 0.8487200140953064


In [4]:
def predict_sentiment(text):
    # This uses the IMDb word index to convert your text to numbers
    word_index = tf.keras.datasets.imdb.get_word_index()
    tokens = text.lower().split()
    # Map words to indices, using 2 for 'unknown' words
    sequence = [word_index.get(w, 2) + 3 for w in tokens] 
    sequence = tf.keras.preprocessing.sequence.pad_sequences([sequence], maxlen=250)
    
    prediction = model.predict(sequence)[0][0]
    sentiment = "Positive 😊" if prediction > 0.5 else "Negative 😞"
    print(f"Review: {text}\nPrediction: {sentiment} ({prediction:.2f})")

# Test it!
predict_sentiment("This movie was an absolute masterpiece with incredible acting.")
predict_sentiment("The plot was boring and the ending made no sense.")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
Review: This movie was not an absolute masterpiece with no incredible acting.
Prediction: Positive 😊 (0.58)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step
Review: The plot was boring and the ending made no sense.
Prediction: Negative 😞 (0.22)
